In [18]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[
    StreamingStdOutCallbackHandler(),
])

chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world-class international chef. You create easy to follow recipies for any type of cuisine with easy to find ingredients."),
    ("human", "I want to cook {cuisine} food.")
])

chef_chain = chef_prompt | chat

In [19]:
veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it."),
    ("human", "{recipe}")
])

veg_chain  = veg_chef_prompt | chat

final_chain = {"recipe": chef_chain} | veg_chain

final_chain.invoke({
    "cuisine": "indian"
})

Great choice! Indian cuisine is known for its bold flavors and aromatic spices. Let's start with a classic and easy recipe for Chicken Tikka Masala.

Ingredients:
- 1 lb boneless, skinless chicken breasts, cut into bite-sized pieces
- 1 cup plain yogurt
- 2 tbsp lemon juice
- 2 tbsp vegetable oil
- 1 onion, finely chopped
- 3 cloves garlic, minced
- 1-inch piece of ginger, grated
- 1 can (14 oz) crushed tomatoes
- 1 tbsp garam masala
- 1 tsp ground cumin
- 1 tsp ground coriander
- 1/2 tsp turmeric
- 1/2 tsp paprika
- 1/2 tsp cayenne pepper (adjust to taste)
- Salt and pepper to taste
- Fresh cilantro, chopped (for garnish)
- Cooked rice or naan bread (for serving)

Instructions:
1. In a bowl, combine the yogurt, lemon juice, 1 tbsp vegetable oil, 1 tbsp garam masala, 1 tsp ground cumin, 1 tsp ground coriander, 1/2 tsp turmeric, 1/2 tsp paprika, 1/2 tsp cayenne pepper, salt, and pepper. Add the chicken pieces and mix well to coat. Marinate for at least 1 hour, or overnight for best resu

AIMessageChunk(content="For a vegetarian version of Chicken Tikka Masala, you can replace the chicken with a plant-based alternative such as tofu or paneer. Here's how you can prepare these alternatives:\n\n1. **Tofu**: \n   - Use firm or extra-firm tofu for this recipe.\n   - Drain the tofu and press it to remove excess water. You can do this by wrapping the tofu block in a clean kitchen towel or paper towels and placing a heavy object on top for about 15-30 minutes.\n   - Cut the tofu into bite-sized cubes and follow the same marinating process as you would with the chicken. Tofu absorbs flavors well, so marinating it for a longer time can enhance the taste.\n\n2. **Paneer**:\n   - Paneer is a type of Indian cheese that holds its shape well when cooked.\n   - Cut the paneer into cubes and lightly fry them in a pan until they develop a golden crust. This step helps the paneer hold its shape better when simmered in the sauce.\n   - You can also marinate the paneer cubes in the yogurt a